# OpenAI Function Calling


# Setup

In [ ]:
import os
import openai
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']

# Functions

## Weather example function

In [ ]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "16",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

## Define a function

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Stuttgart, BW",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

## Messages {.smaller}

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Stuttgart?"
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

## Print response {.smaller}

In [ ]:
print(response)

{
  "id": "chatcmpl-8EzjVmo4zPOq7Z70XEV47HcjF3FBb",
  "object": "chat.completion",
  "created": 1698584585,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Stuttgart\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 17,
    "total_tokens": 98
  }
}
## Response message

In [ ]:
response_message = response["choices"][0]["message"]

In [ ]:
response_message

<OpenAIObject at 0x120dc2c30> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Stuttgart\"\n}"
  }
}

## Response message content

- Content is empty



In [ ]:
response_message["content"]

- Function call is a dictionary



In [ ]:
response_message["function_call"]

- <OpenAIObject at 0x120f9dc10> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Stuttgart\"\n}"
}

## Inspect JSON {.smaller}

In [ ]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Stuttgart'}



In [ ]:
args = json.loads(response_message["function_call"]["arguments"])

In [ ]:
get_current_weather(args)

- '{"location": {"location": "Stuttgart"}, "temperature": "16", "unit": "celsius", "forecast": ["sunny", "windy"]}'


## New message

- New message with no relation to weather

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

## Show response {.smaller}

In [ ]:
print(response)

{
  "id": "chatcmpl-8GqJyabUr85mp4mGjFLZJuKGOoBu6",
  "object": "chat.completion",
  "created": 1699025062,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 10,
    "total_tokens": 85
  }
}



## Use function call auto

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

{
  "id": "chatcmpl-8Ezl8M209h6uqOXuFrGzVwAoSjXVI",
  "object": "chat.completion",
  "created": 1698584686,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 10,
    "total_tokens": 85
  }
}


## Disable function usage {.smaller}

Don't use the function

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8GqKcx0cYK0MWb0ONpWqTTbU1hWlJ",
  "object": "chat.completion",
  "created": 1699025102,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 9,
    "total_tokens": 85
  }
}

## Force function usage {.smaller}

- force to use the function

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8GqKwA2jwWb8PmgrXGZANDyxcsO0l",
  "object": "chat.completion",
  "created": 1699025122,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Stuttgart, BW\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 12,
    "total_tokens": 94
  }
}

## Pass the reults back in the LLM

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

## Response

{
  "id": "chatcmpl-8EzqG43iC6CWCOqCQiSWxngMHJNIQ",
  "object": "chat.completion",
  "created": 1698585004,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 88,
    "completion_tokens": 11,
    "total_tokens": 99
  }
}

## Append message

Append to list of messages

In [ ]:
messages.append(response["choices"][0]["message"])

In [ ]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])

observation = get_current_weather(args)



- This is the response of calling a funtion

In [ ]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

## Response {.smaller}

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)